# Exploring ee Data tutorial

This notebook goes through the Hansen et al 2013 global forest change dataset and translated google earth engine code into Python for visualizing maps: https://developers.google.com/earth-engine/tutorials/tutorial_forest_02?hl=en 

In [1]:
#importing packages
import folium
import geopandas
import shapely
import pyproj
import geojson
import ee

#authentication (commented out after first time)
#ee.Authenticate()

#initializing ee
ee.Initialize()

## Loading custom Map class objects from tutorial

In [2]:
class EngineMap(folium.Map):
    """
    Subclass of folium Map for adding EE images. Here
    self is a folium.Map class instance, and we are adding 
    additional functions to this class type.
    """
    
    def __init__(self, location=None, zoom_start=None, **kwargs):
        # inherit from parent class
        super().__init__(location=location, zoom_start=zoom_start, **kwargs)
    
    
    def add_ee_vector(self, feature, **kwargs):
        """
        Add a google earth engine vector feature to a folium map.
        """
        # create vector feature
        feature = folium.GeoJson(
            data=feature.geometry().getInfo(),
            **kwargs,
        )
        
        # add vector to Map and return self to allow chaining
        self.add_child(feature)
        return self
    
    
    def add_ee_raster(self, image, vis_params={}, **kwargs):
        """
        Add a google earth engine raster layer to a folium map.
        """       
        # handle ImageCollections and Images
        if isinstance(image, ee.ImageCollection):
            ee_image = image.mosaic()

        # get the JSON instructions to show image tiles
        map_id_dict = image.getMapId(vis_params)
        
        # get url of the raster tiles
        tiles = map_id_dict['tile_fetcher'].url_format
        
        # create a folium raster layer and add to self
        raster = folium.raster_layers.TileLayer(
            tiles=tiles,
            attr=kwargs.get("attr", "TODO-attr"),
            name=kwargs.get("name", "test"),
            overlay=True,
            control=True,
        )
        
        # add raster to Map and return self to allow chaining
        self.add_child(raster)
        return self

## Running through the Hansen tutorial

In [3]:
# load a dataset image for the forest change from Hansen (this will serve us for the rest of the tutorial)
gfc2014 = ee.Image('UMD/hansen/global_forest_change_2015')

# make a map on Puerto Rico (El Yunque National Forest)
#      this will also serve us through the tutorial
emap = EngineMap(location=[18.33, -65.82], zoom_start=9)

# add the raster but don't give any visparam stuff to look nice
emap.add_ee_raster(
    image=gfc2014,
    attr="Roí"
)

# display
emap

In [4]:
# add the raster focusing on tree cover 2000 band, still not giving any super nice visuals though
emap.add_ee_raster(
    image=gfc2014,
    attr="Roí",
    vis_params={
        "bands": ['treecover2000'],
    },
)

emap

In [5]:
# adding raster but using Landsat bands 5, 4, 3 (rgb)
emap.add_ee_raster(
    image=gfc2014,
    attr="Roí",
    vis_params={
        "bands": ['last_b50', 'last_b40', 'last_b30'],
    },
)

# display
emap

In [6]:
# adding raster but using bands combining forest loss, tree cover, and forest gain
emap.add_ee_raster(
    image=gfc2014,
    attr="Roí",
    vis_params={
        "bands": ['loss', 'treecover2000', 'gain'],
    },
)

# display
emap

In [7]:
# adding raster but using bands combining forest loss, tree cover, and forest gain
#     and in addition makes it a little nicer by setting value of cover at 255
#     this will make loss red and gain blue

emap.add_ee_raster(
    image=gfc2014,
    attr="Roí",
    vis_params={
        "bands": ['loss', 'treecover2000', 'gain'],
        "max": [1, 255, 1]
    },
)

# display
emap

In [8]:
# adding raster for only forest cover but specifying a color palette
#     this is between black and green I believe
emap.add_ee_raster(
    image=gfc2014,
    attr="Roí",
    vis_params={
        "bands": ['treecover2000'],
        "palette": ['000000','00FF00']
    },
)

# display
emap


In [9]:
# adding raster for only forest cover but specifying a color palette
#     this is between black and green but also specifying max value of 100 which gives it a brighter green
emap.add_ee_raster(
    image=gfc2014,
    attr="Roí",
    vis_params={
        "bands": ['treecover2000'],
        "palette": ['000000','00FF00'],
        "max": 100
    },
)

# display
emap

In [15]:
# adding raster for only forest cover but now instead of black and bright green, 
#     we make nonforest transparent for easier visualization
#     this is achieved with masking!

# creating new map for visualization (older map will still have black background)
emap_2 = EngineMap(location=[18.33, -65.82], zoom_start=9)

# add raster
emap_2.add_ee_raster(
    image=gfc2014.mask(gfc2014), #masking
    attr="Roí",
    vis_params={
        "bands": ['treecover2000'],
        "palette": ['000000','00FF00'],
        "max":100
    },
)

# display
emap_2

In [18]:
# now creating the final map that should look a lot nicer
#    we do this by selecting the bands, loss, cover, gain into their own images
#    then we plot the images each on top of each other 
#    this will give us forest cover in bright green, gain in blue, and loss in red
#    all against a transparent background using masking!

# creating new map
pretty_map = EngineMap(location=[18.33, -65.82], zoom_start=9)

# selecting bands to have 3 images
cover = gfc2014.select('treecover2000') #tree cover
loss = gfc2014.select('loss') #forest loss
gain = gfc2014.select('gain') #forest gain

# now adding raster for each layer sequentially

# cover
pretty_map.add_ee_raster(
    image = cover.updateMask(cover), #masking
    name = 'Forest Cover Puerto Rico 2000', #name
    attr = "Roí", #attribution
    vis_params = {
        "palette": ['000000', '00FF00'], #bright green
        "max": 100
    }
)

# loss
pretty_map.add_ee_raster(
    image = loss.updateMask(loss), #masking
    name = 'Forest Loss Puerto Rico 2000', #name
    attr = "Roí", #attribution
    vis_params = {
        "palette": ['FF0000'] #bright red
    }
)

# gain
pretty_map.add_ee_raster(
    image = gain.updateMask(gain), #masking
    name = 'Forest Gain Puerto Rico 2000', #name
    attr = "Roí", #attribution
    vis_params = {
        "palette": ['0000FF'] #bright blue
    }
)


